# 집값 예측
- 전략
    - 지역구분 $\rightarrow$ 재건축 여부  $\rightarrow$ 크기, 방갯수 $\rightarrow$ 경치
    - grade는 어떻게 쓸지 고민

- Data fields
    - ID : 집을 구분하는 번호
    - date : 집을 구매한 날짜
    - price : 집의 가격(Target variable)
    - bedrooms : 침실의 수
    - bathrooms : 침실 개수 당 화장실의 수(화장실의 수 / 침실의 수 )
    - sqft_living : 주거 공간의 평방 피트(면적)
    - sqft_lot : 부지의 평방 피트(면적)
    - floors : 집의 층 수
    - waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
    - view : 집이 얼마나 좋아 보이는지의 정도
    - condition : 집의 전반적인 상태   #건물 연식이 몇년이 지나면 가격 같음
    - grade : King County grading 시스템 기준으로 매긴 집의 등급   
    - sqft_above : 지하실을 제외한 평방 피트(면적)
    - sqft_basement : 지하실의 평방 피트(면적)   # 지하실 있는 데이터는 따로 취급?
    - yr_built : 지어진 년도
    - yr_renovated : 집을 재건축한 년도
    - zipcode : 우편번호
    - lat : 위도
    - long : 경도
    - sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
    - sqft_lot15 : 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)

In [1]:
import pandas as pd
import numpy as np
import random

In [29]:
# (데이터 수가 작아서 전처리 단계에서는 샘플 안해도 될듯)
# 이중 1000개의 데이터를 임의로 샘플 추출 합니다.  
def rd_data(data, number = 100):
    rd_idx = [random.randint(0,len(data)) for _ in range(number)]   # 0 ~ len(data) 개까지의 인덱스 중 랜덤 샘플 # 개 추출
    sample = data.iloc[rd_idx].reset_index(drop=True)  # 샘플 추출
    return sample

## 0. 데이터 불러오기 및 기본 전처리

In [2]:
house = pd.read_csv("dataset/train.csv")

In [3]:
house_test = pd.read_csv("dataset/test.csv")

In [4]:
house_ref = pd.read_csv("dataset/kc_house_data.csv")

In [5]:
house.drop(columns= ['id'], inplace = True)
house_test.drop(columns= ['id'], inplace = True)
house_ref.drop(columns= ['id'], inplace = True)

In [17]:
# 위도+경도로이루어진 새로운 아이디 부여
house['Newid'] = (round(-house['long']*1000,0)+round(house['lat']/100,6))
house_ref['Newid'] = (round(-house_ref['long']*1000,0)+round(house_ref['lat']/100,6))
#house_test['Newid'] = (-house_test['long']*1000+house_test['lat']/100)

In [18]:
# 'Newid'가 모두 13자리로 잘 만들어졌는지. 아니면 index 출력
def test_Newid(df):
    test = []
    for i in range(len(df)):
        if len(str(df['Newid'].iloc[i]))==13:
            continue
        else:
            test.append(i)
    return test

In [133]:
# 가격, 아이디만 추출
house1 = house[['date', 'price', 'sqft_living', 'Newid']]
house_ref1 = house_ref[['date', 'price', 'sqft_living', 'Newid']]

In [134]:
# Newid가 겹치는 것이 있는지 확인
print(len(house1))
house1.groupby("Newid").size().reset_index(name="counts") .tail()
    #아 겹치는 데이터가있네...

15035


,Newid,counts
14731,122509.474444,1
14732,122511.474285,1
14733,122514.473782,1
14734,122515.473942,1
14735,122519.473477,1


In [144]:
# house_ref에서 house에 있는 데이터만 골라내기위한 Data Frame
# : 위도 경도로 생성된 아이디 +  날짜가 일치하는 데이터를 house_ref로부터 가져오는 함수.
# h_result = pd.DataFrame(columns =['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', \
#                        'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',\
#                        'sqft_living15', 'sqft_lot15', 'Newid'])
def h_result_from_ref(house):
    h_result = pd.DataFrame(columns =['date', 'price', 'sqft_living', 'Newid'])   #데이터 프레임 생성
    for i in range(len(house)):
        temp = house_ref1[house_ref1['Newid'] == house['Newid'].iloc[i]]  #i 번째 아이디가 일치하는지 확인
        if len(temp) != 1 and len(temp) > 0:      #일치하는게 여러개일 경우 # 
            temp = temp[temp['date'] == house['date'].iloc[i]]  # 그 중 날짜 정보까지 일치하는것을 찾는다.
            if len(temp) !=1:
                temp = temp[temp['sqft_living'] == house['sqft_living'].iloc[i]]  # 그 중 날짜 정보까지 일치하는것을 찾는다.
                if len(temp) !=1:
                    temp = temp.iloc[random.randint(0,len(temp)-1)]                
                    h_result = h_result.append(temp)
                else:
                    h_result = h_result.append(temp)
            else:
                h_result = h_result.append(temp)
        elif len(temp) == 1:   #일치하는게 하나일 경우
            h_result = h_result.append(temp)   # 그냥 포함시킨다.
        else:  # house_ref에서 정보 못찾으면 error와 함께 인덱스 정보 출력
            print('error : {}번째 index'.format(i))
  #          h_result = h_result.append(temp)    
    h_result.reset_index(drop=True, inplace=True)
    return h_result

In [149]:
# 크로스 체크 함수
def cross_check(house):
    test = (house['price'] - h_result_from_ref(house)['price']).sum()  #일치 여부 확인. 0이면 정확히 일치한다.
    return test

In [147]:
h_rd = rd_data(house1, 1000)

In [148]:
h_result_from_ref(h_rd).tail()

,date,price,sqft_living,Newid
995,20140820T000000,300000.0,2360,122245.475044
996,20150423T000000,333760.0,1300,122113.476937
997,20141215T000000,850000.0,2700,122024.477187
998,20140916T000000,379000.0,930,122372.476971
999,20140908T000000,285000.0,1890,122041.473613


In [150]:
cross_check(h_rd)

0.0

In [92]:
pd.DataFrame(h_result_from_ref(h_rd).iloc[40]).T

,date,price,Newid
40,20140821T000000,440000,122033


In [93]:
pd.DataFrame(h_rd.iloc[40]).T

,date,price,Newid
40,20140821T000000,475000,122033


In [81]:
(h_rd['price'] - h_result_from_ref(h_rd)['price']).head(43)

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
5          0.0
6          0.0
7          0.0
8          0.0
9          0.0
10         0.0
11         0.0
12         0.0
13         0.0
14         0.0
15         0.0
16         0.0
17         0.0
18         0.0
19         0.0
20         0.0
21         0.0
22         0.0
23         0.0
24         0.0
25         0.0
26         0.0
27         0.0
28         0.0
29         0.0
30         0.0
31         0.0
32         0.0
33         0.0
34         0.0
35         0.0
36         0.0
37         0.0
38         0.0
39         0.0
40     35000.0
41    226000.0
42   -436000.0
Name: price, dtype: float64

In [50]:
cross_check(h_rd)

355000.0

In [79]:
'''
1. 실제 좌표를 구글 맵에 찍어본다
2. zipcode와 입력된 데이터를 몇개 비교해보니 실제 미국 King county에 위치해있는 지역의 집들과 일치한다는 것을 알았다.
3. 만약 실제의 집 데이터를 기반으로 허구의 가격을 매긴다면, 집 값 데이터에 대한 통념적 상식이 안통할 수 있다.
그렇게 되면 데이터 전처리 자체가 불가능하다. 
 ex) 집이 크고, 시내에 있고, 뷰도 좋은데 랜덤 집값을 부여해서 낮은 가격이 책정되었다면 말이 안된다.
4. 반드시 실제 데이터를 기반으로 했을 것이라고 추측할 수 있다.
5. 실제 데이터라면 반드시 구글에 있다.
6. 찾았다. 그런데 데이터 전체 갯수와 id코드들이 다르다.
7. 원 데이터(house_ref)를 받아서 몇 개의 데이터를 제거하고 랜덤하게 섞었을 것이라고 추측했다.
8. 원데이터와 1대1 대응 시켜서 비교할 방법이 없었다. 같은 날짜, 방 갯수 등등은 중복된게 너무 많다.
9. 이 데이터는 1년사이의 거래에 대한 내역이고, 분명 하나의 집은 한번만 거래되었을 것이다. 
(하나의 집이 1년사이에 사고팔고가 두번 일어나는 경우는 흔치않다. 만약 있다면 그거 몇개만 골라서 처리하면 된다.)
10. 따라서 집에 대한 고유의 아이디같은 데이터인 위치데이터를 활용한다.
11. 위도 경도에 대응되는 데이터는 하나씩인것 같다. (확인 필요)
12. 위도 경도를 고유 id라고 생각하고 데이터를 찾아서 대응시킨다.
13. 경도+위도 = id로 변환 :  -'-122.300'*1000 + '47.7263'*1/100 = '-1223.477263'로 만든다.
즉, -경도*1000 + 위도/1000
14. house_ref에서 house데이터에 맞는 것을 추출해서 새로운 데이터 셋을 만든다.
15. 둘의 집값이 일치하는지 체크한다.
16. test셋에도 똑같이 적용한다.
'''
print('Finish')
    

Finish


In [77]:
#pd.concat([house_ref.iloc[aa].T, house.iloc[aa].T], axis=1)

In [ ]:
for i in range(len(house)):

In [8]:
# train 셋을 추출해서 home으로 만들어줍니다.
house.head()  

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


### 기본 전처리
- #id 컬럼 삭제  #id가 정보 식별하려면 계속 따라다녀야할거같아

In [9]:
# date(구매시기) 데이터 줄여주기 ex) 20141013T000000 -> 20141013 ->2014.1013  (구매와 건축 시기 차이 구할 때 유용)
date_2 = [round(int(house['date'].iloc[i][:8])/10000, 4) for i in range(len(house))]
house['date'] = date_2

In [10]:
# price 정규화 0 < price < 1
house['price'] = house['price']/7.700000e+06

---

## 특징별 데이터 분석 및 전처리

- Lable : price : 집의 가격(Target variable)  # 최댓값으로 나눠 정규화함
- 시기적 요소 : 건축 년도, 재건축년도, 구매 날짜 (리만브라더스 등 큰 사건에 따른 영향 고려)
    - date : 집을 구매한 날짜 # T0000.. 없애기 #20140502 ~ 20150514
    - yr_built : 지어진 년도
    - yr_renovated : 집을 재건축한 년도  # 재건축은 시기가 또 달라짐
        - 건물 감가상각 (30~50년 : 대략 40년), 그 이후는 토지가격.
        - 건물가 감가상각고려해서 피팅
- 내적 요소 : 집의 면적, 방 갯수 등
    - bedrooms : 침실의 수 
    - bathrooms : 침실 개수 당 화장실의 수(화장실의 수 / 침실의 수 )
    - floors : 집의 층 수  #1~3.5층까지 있다. 아 단독주택들이구나
    - sqft_above : 지하실을 제외한 평방 피트(면적)
    - sqft_basement : 지하실의 평방 피트(면적)   # 지하실 있는 데이터는 따로 취급?
    - sqft_living : 주거 공간의 평방 피트(면적)
    - sqft_lot : 부지의 평방 피트(면적)   #일단 집의 종류를 분류해야할거같은데
    - sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
    - sqft_lot15 : 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
- 외적 요소 : 뷰, 보이는 상태
    - waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)  # 거의 안보임
    - view : 집이 얼마나 좋아 보이는지의 정도  #0~4점 보통 0점
    - condition : 집의 전반적인 상태   #건물 연식이 몇년이 지나면 가격 같음
- 위치 요소
    - zipcode : 우편번호 #집 위치 판단할때좋을듯. 같은 위치의 분류로 나누기  - 위치와 무관. 삭제
    - lat : 위도  #같은 위치의 분류로 나누기   - 비지도 학습   위치 분류
    - long : 경도  #같은 위치의 분류로 나누기  - 비지도 학습
- 기타 요소
    - grade : King County grading 시스템 기준으로 매긴 집의 등급   
- 영향이 있을까?
    

### 1. zipcode : 삭제
- zipcode는 위치와 상관이 없다.
- 같은 zipcode를 가지는 집들 중 (위도, 경도)를 구글 지도에 찍어보면 서로 완전 다른 지역을 나타내고 있다.
- 따라서 zipcode는 무의미한 정보이다.

In [12]:
# zipcode와 (위도, 경도) 정보 조사
house[['id', 'price', 'zipcode', 'lat', 'long']].sort_values(by = ['zipcode']).reset_index(drop=True).head()

,id,price,zipcode,lat,long
0,9015,0.037662,98001,47.3530,-122.294
1,13162,0.059740,98001,47.2619,-122.271
2,13160,0.033117,98001,47.3318,-122.277
3,14135,0.053896,98001,47.3359,-122.257
4,1557,0.043117,98001,47.3524,-122.285


In [13]:
#zipcode 삭제
house.drop(columns=['zipcode'], inplace=True)

### 2. (위도, 경도)로 구분해보기
- 위도와 경도를 오름차순으로 정렬해서 비슷한 지역 애들로 묵기
- 구글지도에 전부 찍어서 눈으로 보는게 제일 좋은데 어떻게든 가능할듯... 나중에 해보기

In [18]:
house['lat'].max(), house['lat'].min(), house['long'].max(), house['long'].min()

(47.7776, 47.1559, -121.315, -122.519)

In [19]:
#위도 경도 정보만 추출 및 오름차순 정렬
house[['id', 'price',  'lat', 'long', 'yr_built', 'yr_renovated', 'waterfront']].sort_values(by = ['lat', 'long']).reset_index(drop=True).head()

,id,price,lat,long,yr_built,yr_renovated,waterfront
0,2354,0.049351,47.1559,-121.646,1963,0,0
1,9070,0.097403,47.1622,-121.971,1979,0,0
2,8838,0.043506,47.1647,-121.973,1969,0,0
3,286,0.046753,47.1775,-122.022,1974,0,0
4,1969,0.029870,47.1776,-122.011,1919,0,0


In [22]:
# 집값 데이터와 지어진 년도간의 상관계수
round(np.corrcoef(house["price"], house["yr_built"])[0][1], 4), \  #전체 집 값데이터와 지어진 년도 간의 상관계수
round(np.corrcoef(house_no["price"], house_no["yr_built"])[0][1], 4) #No 재건축 데이터와 지어진 년도간의 상관계수

(0.0473, 0.0831)

In [24]:
# 재건축 집값 데이터와 지어진 년도간의 상관계수
round(np.corrcoef(house_re["price"], house_re["yr_built"])[0][1], 4), \ #전체 집 값데이터와 지어진 년도 간의 상관계수
round(np.corrcoef(house_re["price"], house_re["yr_renovated"])[0][1], 4)  #No 재건축 데이터와 지어진 년도간의 상관계수

(0.0651, 0.0968)

### \#. 재건축 여부
- 재건축한 것과 안한 것은 서로 분류해서 생각해야한다.

In [21]:
# 재건축한 것(house_re)과 안한 것(house_no)으로 나누기
house_no = house[house['yr_renovated']==0].reset_index(drop=True)
house_re = house[house['yr_renovated']!=0].reset_index(drop=True)

### \#. 15년 이전 면적 기준, 15년 이후 면적 기준
- 14년 구매엔 이전 것이 적용.
- 15년 구매시엔 15년 것이 적용. 아마도?
- 상관계수를 구해보자.
    - 14년도, 15년 초(몇월까지 ?)에 거래된 집의 가격과 15년 이전 기준 넓이의 상관계수
    - 15년 초 이후에 거래된 집의 가격과 15년 기준 넓이의 상관계수
- 15년 기준과 그 이전 기준 중 고르라고 하면 이전 기준으로 해야할 것 같다.
- 아무래도 부지 넓이랑 가격과는 상관이 없는 것 아닌가 싶다.

In [207]:
house[['id', 'price', 'date', 'sqft_living', 'sqft_living15', 'sqft_lot', 'sqft_lot15', 'yr_renovated']].head()

,id,price,date,sqft_living,sqft_living15,sqft_lot,sqft_lot15,yr_renovated
0,0,0.028818,2014.1013,1180,1340,5650,5650,0
1,1,0.023377,2015.0225,770,2720,10000,8062,0
2,2,0.066234,2015.0218,1680,1800,8080,7503,0
3,3,0.033442,2014.0627,1715,2238,6819,6819,0
4,4,0.037903,2015.0115,1060,1650,9711,9711,0


In [226]:
# 2015이전과 이후로 데이터 구분
criteria = 2015
h_2014 = house[house['date']<criteria]  #2014년도 데이터
h_2015 = house[house['date']>=criteria] #2015년도 데이터

In [239]:
# 주거 공간 넓이랑 가격의 상관계수
h1 = [round(np.corrcoef(h_2014["price"], h_2014["sqft_living"])[0][1], 4), \
round(np.corrcoef(h_2014["price"], h_2014["sqft_living15"])[0][1], 4), \
round(np.corrcoef(h_2015["price"], h_2015["sqft_living"])[0][1], 4), \
round(np.corrcoef(h_2015["price"], h_2015["sqft_living15"])[0][1], 4)]    #상관계수 list ver.
h1_a = np.reshape(np.array(h1),(2,2))  # list to array
pd.DataFrame(h1_a, index=['price 2014', 'price 2015'], columns=["sqft_living", "sqft_living15"]) #DF로 만들기
# 2015 : (0.7149, 0.5851, 0.6772, 0.5899)
# 2015.02 : (0.7133, 0.5899, 0.6762, 0.578)
# 만약 15년 기준과 그 이전 기준 중 고르라고 하면 이전 기준으로 해야할 것 같다.

,sqft_living,sqft_living15
price 2014,0.7149,0.5851
price 2015,0.6772,0.5899


In [240]:
# 부지 넓이랑 가격의 상관계수
h2 = [round(np.corrcoef(h_2014["price"], h_2014["sqft_lot"])[0][1], 4), \
round(np.corrcoef(h_2014["price"], h_2014["sqft_lot15"])[0][1], 4), \
round(np.corrcoef(h_2015["price"], h_2015["sqft_lot"])[0][1], 4), \
round(np.corrcoef(h_2015["price"], h_2015["sqft_lot15"])[0][1], 4)]   #상관계수 list ver.
h2_a = np.reshape(np.array(h2),(2,2))   # list to array
pd.DataFrame(h2_a, index=['price 2014', 'price 2015'], columns=["sqft_lot", "sqft_lot15"]) #DF로 만들기
# (0.0899, 0.0815, 0.1106, 0.0961)
# 아무래도 부지 넓이랑 가격과는 상관이 없는 것 아닌가 싶다.

,sqft_lot,sqft_lot15
price 2014,0.0899,0.0815
price 2015,0.1106,0.0961


### \#. 가격정보

In [126]:
h_yr = house[['price', 'date', 'yr_built', 'yr_renovated']]
h_yr.tail()

,price,date,yr_built,yr_renovated
15030,0.079310,2014.1014,2014,0
15031,0.130844,2015.0326,2009,0
15032,0.046753,2014.0521,2009,0
15033,0.051948,2015.0223,2014,0
15034,0.042208,2014.1015,2008,0


In [128]:
h_yr_sort = h_yr.sort_values(by = ['yr_renovated', 'yr_built', 'date'],ascending=False).reset_index(drop=True)
h_yr_sort.head()

,price,date,yr_built,yr_renovated
0,0.107792,2015.0331,1968,2015
1,0.192857,2015.0506,1964,2015
2,0.105844,2015.0327,1962,2015
3,0.093117,2015.0406,1959,2015
4,0.113312,2015.0501,1956,2015
